In [13]:
from cylinder_fdm_3d import CylinderFDM
from icecream import ic
import numpy as np
from scipy.sparse.linalg import eigsh, cg, gmres
from time import time
from matplotlib import pyplot as plt
from fft_tdse.simulator import LaserPulse
from erfgau import ErfgauPotential
from rich.console import Console
from rich.table import Table
import timeit



In [22]:
n_r = 100
n_z = 200
n_theta = 4
r_max = 20
z_max = 20
solver = CylinderFDM(r_max, z_max, n_r, n_z, n_theta)
L_vec = solver.get_laplacian_fast()
rr, zz = solver.get_rz_meshgrid()

u = rr**.5 * np.exp(-(rr**2 - zz**2)/2)
rhs = -4*np.pi*u.flatten()



ic| self.m_i: array([ 0.,  1., -2., -1.])
ic| cylinder_fdm_3d.py:169 in __init__() at 13:00:19.882
ic| self.n_r: 100, self.n_z: 200, self.n_m: 4
ic| self.r_max: 20, self.z_max: 20
ic| self.n_dof: 80000


In [36]:
def solve_timing(solve, n_r, n_z, m, number=20):
    # n_r = 100
    # n_z = 200
    n_theta = 2*m + 1
    r_max = 20
    z_max = 20
    solver = CylinderFDM(r_max, z_max, n_r, n_z, n_theta)
    L_vec = solver.get_laplacian_fast()
    rr, zz = solver.get_rz_meshgrid()

    u = rr**(np.abs(m) + .5) * np.exp(-(rr**2 - zz**2)/2)
    rhs = -4*np.pi*u.flatten()

    ic('system dimension = ', L_vec[0].shape[0])
    code = lambda: solve(L_vec[0], rhs)
    time = timeit.timeit(code, number=number)
    time_per_solve = time/20
    ic(time_per_solve)
    return time_per_solve

In [50]:
#solve = lambda A, b: cg(A, b, tol=1e-10, maxiter=1000)

def timing_table(solve):

    n_list = np.array([100, 200, 400])
    m_list = np.array([0, 1, 4])

    times = np.zeros((len(n_list), len(m_list)))
    for j,n in enumerate(n_list):
        for i,m in enumerate(m_list):
            times[j,i] = solve_timing(solve, n, n, m, number=10)

    table = Table('Timings')
    for m in m_list:
        table.add_column(f'm = {m}')
    for j,n in enumerate(n_list):
        table.add_row(f'n = {n}', *map(str,times[j]))

    display(table)
    return times, table


In [51]:
solve = lambda A, b: cg(A, b, tol=1e-10, maxiter=1000)
timing_table(solve)


ic| self.m_i: array([0.])
ic| cylinder_fdm_3d.py:169 in __init__() at 13:18:47.101
ic| self.n_r: 100, self.n_z: 100, self.n_m: 1
ic| self.r_max: 20, self.z_max: 20
ic| self.n_dof: 10000
ic| 'system dimension = ', L_vec[0].shape[0]: 10000
ic| time_per_solve: 0.014824270794633776
ic| self.m_i: array([ 0.,  1., -1.])
ic| cylinder_fdm_3d.py:169 in __init__() at 13:18:47.436
ic| self.n_r: 100, self.n_z: 100, self.n_m: 3
ic| self.r_max: 20, self.z_max: 20
ic| self.n_dof: 30000
ic| 'system dimension = ', L_vec[0].shape[0]: 10000
ic| time_per_solve: 0.013676529144868254
ic| self.m_i: array([ 0.,  1.,  2.,  3.,  4., -4., -3., -2., -1.])
ic| cylinder_fdm_3d.py:169 in __init__() at 13:18:47.753
ic| self.n_r: 100, self.n_z: 100, self.n_m: 9
ic| self.r_max: 20, self.z_max: 20
ic| self.n_dof: 90000
ic| 'system dimension = ', L_vec[0].shape[0]: 10000
ic| time_per_solve: 0.013347879203502088
ic| self.m_i: array([0.])
ic| cylinder_fdm_3d.py:169 in __init__() at 13:18:48.099
ic| self.n_r: 200, self.n_z:

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timings ┃ m = 0                ┃ m = 1                ┃ m = 4                ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ n = 100 │ 0.014824270794633776 │ 0.013676529144868254 │ 0.013347879203502088 │
│ n = 200 │ 0.1798997958539985   │ 0.22754063749453052  │ 0.21507504375185818  │
│ n = 400 │ 0.9035920478985645   │ 0.9684639250044711   │ 1.0313564270501956   │
└─────────┴──────────────────────┴──────────────────────┴──────────────────────┘

(array([[0.01482427, 0.01367653, 0.01334788],
        [0.1798998 , 0.22754064, 0.21507504],
        [0.90359205, 0.96846393, 1.03135643]]),
 <rich.table.Table at 0x1375cefb0>)